<a href="https://colab.research.google.com/github/Khushi-MA/trialreu1/blob/main/sept11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/REU/reuwork/trialreu1
!pwd
!ls

/content/drive/MyDrive/REU/reuwork/trialreu1
/content/drive/MyDrive/REU/reuwork/trialreu1
Data1  labels.pt  main.ipynb  README.md  sept7.ipynb  video_data.pt


In [4]:
!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [64]:
# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# initial mediapipe only palma nd fingers

In [7]:
# # Function to blackout the background and keep only the hand movements
# def blackout_background(video_path, output_path):
#     hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
#     cap = cv2.VideoCapture(video_path)

#     # Get video details
#     frame_width = int(cap.get(3))
#     frame_height = int(cap.get(4))
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')

#     # Video writer to save the processed video
#     out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Convert the image to RGB as MediaPipe expects this format
#         image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#         # Get hand landmarks
#         result = hands.process(image_rgb)

#         # Create a blackout frame (all black)
#         black_frame = np.zeros_like(frame)

#         if result.multi_hand_landmarks:
#             for hand_landmarks in result.multi_hand_landmarks:
#                 # Draw the hand landmarks on the black frame
#                 mp_drawing.draw_landmarks(
#                     black_frame,
#                     hand_landmarks,
#                     mp_hands.HAND_CONNECTIONS,
#                     mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
#                 )

#         # Write the black_frame with only hands visible to output video
#         out.write(black_frame)

#     cap.release()
#     out.release()
#     hands.close()

# estimated hands from wrist to down

In [13]:
# Function to extend wrist to elbow approximation
def extend_wrist_to_elbow(black_frame, hand_landmarks):
    # Get wrist landmark (landmark 0 in MediaPipe's hand model)
    wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]

    # Calculate the elbow position by extending from wrist along the vector
    # Here we are estimating the elbow to be at a certain distance below the wrist
    wrist_x = int(wrist.x * black_frame.shape[1])
    wrist_y = int(wrist.y * black_frame.shape[0])

    # Arbitrary length to extend for the 'elbow'
    length = 100  # You can adjust this based on how long you want the arm to be

    # Calculate 'elbow' position
    elbow_x = wrist_x
    elbow_y = wrist_y + length

    # Draw the line from wrist to the 'elbow'
    cv2.line(black_frame, (wrist_x, wrist_y), (elbow_x, elbow_y), (255, 255, 255), thickness=2)

In [14]:
# Function to blackout the background and keep only the hand movements
def blackout_background(video_path, output_path):
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    cap = cv2.VideoCapture(video_path)

    # Get video details
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Video writer to save the processed video
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the image to RGB as MediaPipe expects this format
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get hand landmarks
        result = hands.process(image_rgb)

        # Create a blackout frame (all black)
        black_frame = np.zeros_like(frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw the hand landmarks on the black frame
                mp_drawing.draw_landmarks(
                    black_frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
                )
                # Extend the wrist to elbow
                extend_wrist_to_elbow(black_frame, hand_landmarks)

        # Write the black_frame with only hands visible to output video
        out.write(black_frame)

    cap.release()
    out.release()
    hands.close()

In [15]:
# Example usage: Process the dictionary videos
dictionary_videos_path = "/content/drive/MyDrive/REU/reuwork/trialreu1/Data1/Words"  # Replace with actual path
blackout_videos_output_path = "/content/drive/MyDrive/REU/reuwork/trialreu1/Data1/black1"  # Replace with actual output folder

# Iterate over all dictionary videos and process each
for video_file in os.listdir(dictionary_videos_path):
    if video_file.endswith(".mp4"):  # Process only .mp4 videos (adjust if necessary)
        video_path = os.path.join(dictionary_videos_path, video_file)
        output_path = os.path.join(blackout_videos_output_path, video_file)
        blackout_background(video_path, output_path)

print("All dictionary videos processed and saved in the blackout dataset.")


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


All dictionary videos processed and saved in the blackout dataset.


# with face

In [100]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [101]:


# Initialize MediaPipe Hands and Face Mesh models
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

In [102]:
# Function to extend wrist to elbow approximation
def extend_wrist_to_elbow(black_frame, hand_landmarks):
    # Get wrist landmark (landmark 0 in MediaPipe's hand model)
    wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]

    # Calculate the elbow position by extending from wrist along the vector
    wrist_x = int(wrist.x * black_frame.shape[1])
    wrist_y = int(wrist.y * black_frame.shape[0])

    # Arbitrary length to extend for the 'elbow'
    length = 100  # You can adjust this based on how long you want the arm to be

    # Calculate 'elbow' position
    elbow_x = wrist_x
    elbow_y = wrist_y + length

    # Draw the line from wrist to the 'elbow'
    cv2.line(black_frame, (wrist_x, wrist_y), (elbow_x, elbow_y), (255, 255, 255), thickness=2)

In [103]:
# Function to blackout the background and keep only hand and face landmarks
def blackout_background(video_path, output_path):
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
    cap = cv2.VideoCapture(video_path)

    # Get video details
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Video writer to save the processed video
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the image to RGB as MediaPipe expects this format
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get hand landmarks
        result_hands = hands.process(image_rgb)

        # Get face landmarks
        result_face = face_mesh.process(image_rgb)

        # Create a blackout frame (all black)
        black_frame = np.zeros_like(frame)

        # Draw hands if found
        if result_hands.multi_hand_landmarks:
            for hand_landmarks in result_hands.multi_hand_landmarks:
                # Draw the hand landmarks on the black frame
                mp_drawing.draw_landmarks(
                    black_frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
                )
                # Extend the wrist to elbow
                extend_wrist_to_elbow(black_frame, hand_landmarks)

        # Draw face if found
        if result_face.multi_face_landmarks:
            for face_landmarks in result_face.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame,
                    face_landmarks,
                    mp_face_mesh.FACEMESH_CONTOURS,
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1),
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1)
                )

        # Write the black_frame with only hands and face visible to output video
        out.write(black_frame)

    cap.release()
    out.release()
    hands.close()
    face_mesh.close()

In [104]:

# Example usage: Process the dictionary videos
dictionary_videos_path = "/content/drive/MyDrive/REU/reuwork/trialreu1/Data1/Words"  # Replace with actual path
blackout_videos_output_path = "/content/drive/MyDrive/REU/reuwork/trialreu1/Data1/black1"  # Replace with actual output folder

# Iterate over all dictionary videos and process each
for video_file in os.listdir(dictionary_videos_path):
    if video_file.endswith(".mp4"):  # Process only .mp4 videos (adjust if necessary)
        video_path = os.path.join(dictionary_videos_path, video_file)
        output_path = os.path.join(blackout_videos_output_path, video_file)
        blackout_background(video_path, output_path)

print("All dictionary videos processed and saved in the blackout dataset.")


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


All dictionary videos processed and saved in the blackout dataset.


# frame extraction

In [5]:
# Step 1: Frame Extraction
def extract_frames(video_path, frame_rate=1):
    print(f"Extracting frames from {video_path}...")
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frames.append(frame)
        count += 1
    cap.release()
    print(f"Extracted {len(frames)} frames from {video_path}.")
    return frames

# Step 2: Normalization & Resizing
def preprocess_frame(frame, size=(224, 224)):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    frame = transform(frame)
    # print("Frame preprocessed.")
    return frame

# Step 3: Augmentation (optional)
def augment_frames(frames):
    print("Augmenting frames...")
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0))
    ])
    augmented_frames = []
    for frame in frames:
        frame = transform(frame)
        augmented_frames.append(frame)
    print(f"Augmented {len(augmented_frames)} frames.")
    return augmented_frames

# Step 4: Label Assignment
def get_label_from_filename(filename):
    label = filename.split()[1].split('.')[0]
    print(f"Assigned label '{label}' to {filename}.")
    return label

# Main Preprocessing Function
def preprocess_videos(directory):
    print(f"Preprocessing videos in directory: {directory}")
    video_data = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):
            video_path = os.path.join(directory, filename)
            frames = extract_frames(video_path)
            frames = [preprocess_frame(frame) for frame in frames]
            frames = augment_frames(frames)  # Optional
            label = get_label_from_filename(filename)
            video_data.append(frames)
            labels.append(label)
    print("Preprocessing complete.")
    return video_data, labels



In [6]:
import torch
from torchvision import transforms


# Example usage
video_directory = 'Data1/black1'
video_data, labels = preprocess_videos(video_directory)

# Save preprocessed data for later use
torch.save(video_data, 'video_data.pt')
torch.save(labels, 'labels.pt')
print("Data saved.")

Preprocessing videos in directory: Data1/black1
Extracting frames from Data1/black1/a0 butcher.mp4...
Extracted 58 frames from Data1/black1/a0 butcher.mp4.
Augmenting frames...
Augmented 58 frames.
Assigned label 'butcher' to a0 butcher.mp4.
Extracting frames from Data1/black1/a1 aadhar_card.mp4...
Extracted 94 frames from Data1/black1/a1 aadhar_card.mp4.
Augmenting frames...
Augmented 94 frames.
Assigned label 'aadhar_card' to a1 aadhar_card.mp4.
Preprocessing complete.
Data saved.


# labelling

In [7]:
# Create a label mapping
unique_labels = list(set(labels))
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

# Convert string labels to numerical labels
numerical_labels = [label_to_index[label] for label in labels]

In [8]:
import torch
from torch.utils.data import Dataset

class SignDataset(Dataset):
    def __init__(self, video_data, labels, transform=None, max_frames=100):
        self.video_data = video_data
        self.labels = labels
        self.transform = transform
        self.max_frames = max_frames

    def __len__(self):
        return len(self.video_data)

    def __getitem__(self, idx):
        video = self.video_data[idx]
        label = self.labels[idx]

        # Pad or truncate video to max_frames
        if len(video) < self.max_frames:
            padding = [torch.zeros_like(video[0])] * (self.max_frames - len(video))
            video.extend(padding)
        else:
            video = video[:self.max_frames]

        if self.transform:
            video = [self.transform(frame) for frame in video]
        video = torch.stack(video)
        label = torch.tensor(label, dtype=torch.long)  # Ensure label is a tensor of type long
        return video, label

# model architecture

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models


In [10]:
class SignRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(SignRecognitionModel, self).__init__()
        # Use a pre-trained 2D-CNN (e.g., ResNet) as the feature extractor
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()  # Remove the final fully connected layer

        # LSTM to capture temporal relationships
        self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=2, batch_first=True)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        print("Forward pass started.")
        batch_size, seq_length, c, h, w = x.size()
        cnn_out = []

        # Pass each frame through the CNN
        for t in range(seq_length):
            frame = x[:, t, :, :, :]
            cnn_out.append(self.cnn(frame))
            print(f"Processed frame {t+1}/{seq_length} through CNN.")

        # Stack the CNN outputs and pass through LSTM
        cnn_out = torch.stack(cnn_out, dim=1)
        print("Stacked CNN outputs.")
        lstm_out, _ = self.lstm(cnn_out)
        print("Passed through LSTM.")

        # Take the output of the last LSTM cell
        lstm_out = lstm_out[:, -1, :]
        print("Extracted output from the last LSTM cell.")

        # Pass through the fully connected layer
        out = self.fc(lstm_out)
        print("Passed through the fully connected layer.")
        print("Forward pass completed.")
        return out

# query processing

In [11]:
import os
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity


In [12]:
import torch

# Load preprocessed data and labels
video_data = torch.load('video_data.pt', weights_only=True)
labels = torch.load('labels.pt', weights_only=True)

print("Data and labels loaded.")

Data and labels loaded.


In [13]:
import os

# Function to perform exact match
def exact_match(query, labels):
    if query in labels:
        index = labels.index(query)
        return video_data[index]
    else:
        return None


In [14]:


# Function to get BERT embeddings for a word
def get_bert_embedding(word):
    inputs = tokenizer(word, return_tensors='pt')
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Function to find the most similar word using BERT
def similar_word_search(query, labels):
    query_embedding = get_bert_embedding(query)
    label_embeddings = [get_bert_embedding(label) for label in labels]

    similarities = [cosine_similarity(query_embedding, label_embedding)[0][0] for label_embedding in label_embeddings]
    max_similarity_index = np.argmax(similarities)

    if similarities[max_similarity_index] > 0.7:  # Threshold for similarity
        return labels[max_similarity_index]
    else:
        return None



In [15]:
import torch

# Load the labels from the .pt file
labels_path = "labels.pt"  # Replace with the actual path to labels.pt
labels = torch.load(labels_path)

# Print the loaded labels
print(labels)


['butcher', 'aadhar_card']


<ipython-input-15-086fb1bc4ec2>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(labels_path)


In [16]:
# from transformers import BertTokenizer, BertModel
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Load BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')

# # Function to get BERT embeddings for a word
# def get_bert_embedding(word):
#     inputs = tokenizer(word, return_tensors='pt')
#     outputs = bert_model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# # Function to find exact match for the word in the dictionary
# def exact_match(query, labels):
#     if query in labels:
#         # Return corresponding video frames for the exact match
#         return query  # Placeholder for actual frame retrieval logic
#     return None

# # Function to search for similar words using BERT embeddings
# def similar_word_search(query, labels):
#     query_embedding = get_bert_embedding(query)

#     # Find the embeddings for all dictionary labels
#     label_embeddings = {label: get_bert_embedding(label) for label in labels}

#     # Calculate cosine similarities between the query and each label
#     similarities = {label: cosine_similarity(query_embedding, label_embedding)[0][0]
#                     for label, label_embedding in label_embeddings.items()}

#     # Find the most similar word based on cosine similarity
#     similar_word = max(similarities, key=similarities.get)

#     # Set a threshold for similarity to ensure the words are closely related
#     if similarities[similar_word] > 0.7:  # Adjust threshold if necessary
#         return similar_word
#     return None

# # Example usage
# query = "butcher"
# labels = ["chef", "cook", "carpenter", "butcher"]  # Example dictionary of words

query = "butcher"

video_frames = exact_match(query, labels)

if not video_frames:
    print(f"Exact match not found for '{query}'. Searching for similar words...")
    similar_word = similar_word_search(query, labels)
    if similar_word:
        video_frames = exact_match(similar_word, labels)
        print(f"Similar word found: '{similar_word}', corresponding video frames retrieved.")
    else:
        print("No similar word found.")
else:
    print(f"Exact match found for '{query}', corresponding video frames retrieved.")


Exact match found for 'butcher', corresponding video frames retrieved.


In [17]:
def extract_features_from_frames(frames, model, max_frames=100):
    preprocessed_frames = [preprocess_frame(frame) for frame in frames]

    # Pad or truncate frames to max_frames
    if len(preprocessed_frames) < max_frames:
        padding = [torch.zeros_like(preprocessed_frames[0])] * (max_frames - len(preprocessed_frames))
        preprocessed_frames.extend(padding)
    else:
        preprocessed_frames = preprocessed_frames[:max_frames]

    preprocessed_frames = torch.stack(preprocessed_frames).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        features = model(preprocessed_frames)

    return features

# feature extraction forom retrived video

In [18]:
import torch
import torch.nn as nn
from torchvision import models


In [19]:

class SignRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(SignRecognitionModel, self).__init__()
        # Use a pre-trained 2D-CNN (e.g., ResNet) as the feature extractor
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()  # Remove the final fully connected layer

        # LSTM to capture temporal relationships
        self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=2, batch_first=True)

        # Fully connected layer for classification
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.size()
        cnn_out = []

        # Pass each frame through the CNN
        for t in range(seq_length):
            frame = x[:, t, :, :, :]
            cnn_out.append(self.cnn(frame))

        # Stack the CNN outputs and pass through LSTM
        cnn_out = torch.stack(cnn_out, dim=1)
        lstm_out, _ = self.lstm(cnn_out)

        # Take the output of the last LSTM cell
        lstm_out = lstm_out[:, -1, :]

        # Pass through the fully connected layer
        out = self.fc(lstm_out)
        return out


In [20]:
import torch
import torch.nn as nn
from torchvision import models
# import the necessary module for DataLoader
from torch.utils.data import DataLoader # import DataLoader
from torchvision import transforms


# Prepare Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

max_frames = 100  # Set the maximum number of frames for each video
dataset = SignDataset(video_data, numerical_labels, transform=transform, max_frames=max_frames)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [21]:
# Initialize model, loss function, and optimizer
num_classes = len(unique_labels)
model = SignRecognitionModel(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create the directory to save the model if it doesn't exist
save_dir = '/content/trialreu1'
os.makedirs(save_dir, exist_ok=True)

# Training loop with model saving at the end of training
def train_model(model, dataloader, criterion, optimizer, num_epochs=2, save_path=None):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        print(f"Starting epoch {epoch + 1}/{num_epochs}")

        for batch_idx, (inputs, labels) in enumerate(dataloader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx}/{len(dataloader)} processed. Current loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {epoch_loss:.4f}")

    # Save the trained model at the specified path
    if save_path:
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")

# Set the save path to the content/model folder
save_path = os.path.join(save_dir, 'sign_recognition_model1.pth')

# Train the model and save it
train_model(model, dataloader, criterion, optimizer, num_epochs=2, save_path=save_path)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 115MB/s]


Starting epoch 1/2
Batch 0/1 processed. Current loss: 0.6886
Epoch 1/2 completed. Loss: 0.6886
Starting epoch 2/2
Batch 0/1 processed. Current loss: 0.5480
Epoch 2/2 completed. Loss: 0.5480
Model saved to /content/trialreu1/sign_recognition_model1.pth


# 5. Spotting in Continuous Video (Temporal Localization)

Summary

Input Continuous Video: Extract frames and preprocess them.

Feature Extraction: Use the same feature extractor (CNN+LSTM) to extract features from the continuous video.

Sliding Window Approach: Slide a window over the continuous video to extract features for each segment.

Similarity Search: Compute similarity scores between the segment features and the retrieved video's features.

Localization: Predict the time interval in the continuous video where the sign appears.

In [40]:
import cv2
import mediapipe as mp
from torchvision import transforms
import numpy as np
from PIL import Image

# MediaPipe setup
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
face_mesh = mp_face.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Function to blackout background and keep only hands and face
def blackout_background_with_hands_and_face(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(frame_rgb)
    results_face = face_mesh.process(frame_rgb)

    # Create an all-black frame of the same size
    black_frame = np.zeros_like(frame)

    # Draw hand landmarks on the black frame
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                black_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Draw face landmarks on the black frame
    if results_face.multi_face_landmarks:
        for face_landmarks in results_face.multi_face_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                black_frame, face_landmarks, mp_face.FACEMESH_CONTOURS)

    return black_frame

# Step 1: Input Continuous Video - Extract frames from the continuous video, apply blackout, and save the video
def extract_and_blackout_frames(video_path, output_path, frame_rate=1):
    print(f"Extracting and processing frames from {video_path}...")
    cap = cv2.VideoCapture(video_path)

    # Get frame dimensions
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get the original video's FPS

    # Set up VideoWriter to save the processed frames into a video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            # Blackout background for the current frame (keep only hands and face)
            processed_frame = blackout_background_with_hands_and_face(frame)
            out.write(processed_frame)  # Write the processed frame to the video
        count += 1

    cap.release()
    out.release()
    print(f"Extracted and processed frames saved to {output_path}.")

# Example usage:
video_path = '/content/drive/MyDrive/REU/reuwork/trialreu1/Data1/v0.mp4'
output_path = '/content/drive/MyDrive/REU/reuwork/trialreu1/Data1/black.mp4'
extract_and_blackout_frames(video_path, output_path, frame_rate=1)


Extracting and processing frames from /content/drive/MyDrive/REU/reuwork/trialreu1/Data1/v0.mp4...
Extracted and processed frames saved to /content/drive/MyDrive/REU/reuwork/trialreu1/Data1/black.mp4.


In [41]:
# Step 2: Feature Extraction
# Use the same feature extractor (CNN+LSTM) to extract features from the continuous video.

def extract_features_from_video(video_path, model, max_frames=100):
    frames = extract_frames(video_path)
    preprocessed_frames = [preprocess_frame(frame) for frame in frames]

    # Pad or truncate frames to max_frames
    if len(preprocessed_frames) < max_frames:
        padding = [torch.zeros_like(preprocessed_frames[0])] * (max_frames - len(preprocessed_frames))
        preprocessed_frames.extend(padding)
    else:
        preprocessed_frames = preprocessed_frames[:max_frames]

    preprocessed_frames = torch.stack(preprocessed_frames).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        features = model(preprocessed_frames)

    return features

In [42]:
# Step 3: Sliding Window Approach
# Slide a window over the continuous video to extract features for each segment.
def sliding_window_features(video_path, model, window_size=30, stride=15):
    frames = extract_frames(video_path)
    preprocessed_frames = [preprocess_frame(frame) for frame in frames]
    num_frames = len(preprocessed_frames)

    window_features = []
    for start in range(0, num_frames - window_size + 1, stride):
        window = preprocessed_frames[start:start + window_size]
        window = torch.stack(window).unsqueeze(0)  # Add batch dimension

        with torch.no_grad():
            features = model(window)

        window_features.append((start, features))

    return window_features


In [43]:
# Step 4: Similarity Search
# Compute similarity scores between the segment features and the retrieved video's features.

from sklearn.metrics.pairwise import cosine_similarity


def compute_similarity(retrieved_features, window_features):
    similarities = []
    for start, features in window_features:
        similarity = cosine_similarity(retrieved_features, features)
        similarities.append((start, similarity))
    return similarities

In [44]:
# Step 5: Localization
# Predict the time interval in the continuous video where the sign appears based on the similarity scores.

def find_best_match(similarities, threshold=0.8):
    best_match = None
    best_score = -1
    for start, similarity in similarities:
        score = similarity[0][0]
        if score > best_score and score > threshold:
            best_score = score
            best_match = start
    return best_match, best_score

In [45]:

# Example usage
continuous_video_path = output_path

# The video is now saved as 'blackcont.mp4' with only hands and face visible.

# Extract features from the retrieved video frames
if video_frames:
    retrieved_features = extract_features_from_frames(video_frames, model)
    if retrieved_features is None:
        print("Failed to extract features from the retrieved video frames.")
    else:
        # Extract features from the continuous video using sliding window
        window_features = sliding_window_features(continuous_video_path, model)

        # Compute similarity scores
        similarities = compute_similarity(retrieved_features, window_features)

        # Find the best match
        best_match, best_score = find_best_match(similarities)

        if best_match is not None:
            print(f"Sign found at timestamp: {best_match} frames with similarity score: {best_score:.4f}")
        else:
            print("Sign not found in the continuous video.")
else:
    print("No video frames retrieved for the query.")

Extracting frames from /content/drive/MyDrive/REU/reuwork/trialreu1/Data1/black.mp4...
Extracted 149 frames from /content/drive/MyDrive/REU/reuwork/trialreu1/Data1/black.mp4.
Sign not found in the continuous video.
